In [1]:
import yaml
import requests

BASE_URL = "https://circleci.com/api/v1.1/"

def load_cfg():
    fcfg = open("../config.yaml")
    return yaml.load(fcfg)
    
cfg = load_cfg()
token = cfg['circleci']['api-token']
# print token

In [2]:
import requests
from requests.auth import HTTPBasicAuth

r = requests.get(BASE_URL + 'me', auth=HTTPBasicAuth(token, ''))
rjson = r.json()
print rjson['all_emails'][0]

maxwunj@gmail.com


In [3]:
import json

r = requests.get(BASE_URL + 'projects', auth=HTTPBasicAuth(token, ''))
# print r.text
rjson = r.json()
# print json.dumps(rjson[0], indent = 4)
reponame = rjson[0]['reponame']
vcs_type =  rjson[0]['vcs_type']
username = rjson[0]['username']

In [4]:
# GET: /project/:vcs-type/:username/:project
r = requests.get(BASE_URL + 'project' + 
                 '/' + vcs_type + 
                 '/' + username +
                 '/' + reponame
                 , auth=HTTPBasicAuth(token, ''))
# print r.text
build_num = r.json()[0]['build_num']
print build_num

67


In [12]:
import xml.etree.cElementTree as ET

def get_xunit_from_build_id(build_id = 67):
    # GET: /project/:vcs-type/:username/:project/:build_num/artifacts
    r = requests.get(BASE_URL + 'project' + 
                     '/' + vcs_type + 
                     '/' + username +
                     '/' + reponame +
                     '/' + str(build_id) + 
                     '/artifacts'
                     , auth=HTTPBasicAuth(token, ''))

    # print r.text
    for artifact in r.json():
        print artifact['url']
        res = r = requests.get(artifact['url'], stream=True)
        # The fetched XML files are XUnit format test history.
        # print res.text
        tree = ET.fromstring(res.text)
        for elem in tree.iter('testcase'):
            suite = elem.get('classname')
            case = elem.get('name')
            time = elem.get('time')
            passed = True
            for child in elem:
                if 'failure' == child.tag:
                    passed = False
            print "suite=%s, case=%s, pass=%s" %(suite, case, passed)

get_xunit_from_build_id(build_num)

https://67-77958022-gh.circle-artifacts.com/0/tmp/circle-junit.isyeuXm/junit/TEST-org.maxwu.jrefresh.selenium.TemperatureConverterTest.xml
suite=Scenario: Open Temperature Convertor, case=Given Web browser initialized, pass=True
suite=Scenario: Open Temperature Convertor, case=Given Google Entrance Page with:, pass=True
suite=Scenario: Open Temperature Convertor, case=When Search "temperature converter", pass=True
suite=Scenario: Open Temperature Convertor, case=Then The page title is "temperature converter", pass=True
suite=Scenario: Open Temperature Convertor, case=And There is a "Temperature" option selected, pass=True
suite=Scenario: Open Temperature Convertor, case=Scenario: Open Temperature Convertor, pass=True
suite=Scenario: Calculate temperature degrees, case=Given Temperature background check, pass=True
suite=Scenario: Calculate temperature degrees, case=Given "Fahrenheit" select is present, pass=True
suite=Scenario: Calculate temperature degrees, case=And "Celsius" select is